In [ ]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img
import numpy as np

# Load the image
input_image = load_img("photo.jpg")
input_image_array = img_to_array(input_image)
#print(input_image_array)
# Get the dimensions of the input image
rows, cols, channels = input_image_array.shape

print(rows,cols,channels)



In [ ]:
#THIS IS FOR TRAINING AND TESTING MODEL

In [2]:
import os

base_path = './data/'
train_path = './train/'
test_path = './test/'
# Create the folders
os.makedirs(train_path + 'vehicles', exist_ok=True)
os.makedirs(train_path + 'non-vehicles', exist_ok=True)
os.makedirs(test_path + 'vehicles', exist_ok=True)
os.makedirs(test_path + 'non-vehicles', exist_ok=True)

In [3]:
import shutil

# Define the source and destination paths
src_vehicles_path = base_path + 'vehicles/'
src_non_vehicles_path = base_path + 'non-vehicles/'

dest_train_vehicles_path = train_path + 'vehicles/'
dest_train_non_vehicles_path = train_path + 'non-vehicles/'
dest_test_vehicles_path = test_path + 'vehicles/'
dest_test_non_vehicles_path = test_path + 'non-vehicles/'

#split the data into 80% for training and 20% for testing
split_ratio = 0.8

# List all vehicle and non-vehicle images
vehicle_images = os.listdir(src_vehicles_path)
non_vehicle_images = os.listdir(src_non_vehicles_path)

# Calculate the number of images for training and testing
num_train_vehicles = int(len(vehicle_images) * split_ratio)
num_train_non_vehicles = int(len(non_vehicle_images) * split_ratio)

# Copy images to the training folders
for img in vehicle_images[:num_train_vehicles]:
    shutil.copy(src_vehicles_path + img, dest_train_vehicles_path + img)

for img in non_vehicle_images[:num_train_non_vehicles]:
    shutil.copy(src_non_vehicles_path + img, dest_train_non_vehicles_path + img)

# Copy the remaining images to the testing folders
for img in vehicle_images[num_train_vehicles:]:
    shutil.copy(src_vehicles_path + img, dest_test_vehicles_path + img)

for img in non_vehicle_images[num_train_non_vehicles:]:
    shutil.copy(src_non_vehicles_path + img, dest_test_non_vehicles_path + img)


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.metrics import Precision, Recall, F1Score
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.applications import  MobileNetV2
from tensorflow.keras.regularizers import l2
import seaborn as sns
from tensorflow.keras import models, layers

In [9]:
Batch_size1 = 32
# Define data augmentation parameters for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5], 
    channel_shift_range=50 #shifts the color channels 
)
# Create a data generator for test data
test_datagen = ImageDataGenerator(rescale=1./255
)

test_Image = test_datagen.flow_from_directory(
    './test',
    target_size=(1500, 1000),
    batch_size=Batch_size1,
    class_mode='binary',
    shuffle=False  # Important: Set shuffle to False for evaluation
)

train_Image = train_datagen.flow_from_directory(
    './train',
    target_size=(1500, 1000),
    batch_size=Batch_size1,
    class_mode='binary'
)

Found 3553 images belonging to 2 classes.
Found 14207 images belonging to 2 classes.


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D,UpSampling2D, Conv2DTranspose
from keras.optimizers import Adam

In [10]:
classifier2 = Sequential() # initialize neural network
        # Convolutional layers
classifier2.add(Conv2D(3, kernel_size=(3,3),strides=(1, 1), input_shape=(1500, 1000, 3), activation='PReLU',padding='same'))
        #classifier.add(MaxPooling2D(pool_size=(2, 2)))
        
classifier2.add(Conv2D(35, kernel_size=(3,3),strides=(1, 1), activation='PReLU',padding='same'))
        #classifier.add(MaxPooling2D(pool_size=(2, 2)))
        
classifier2.add(Conv2D(35, kernel_size=(3,3),strides=(1, 1), activation='PReLU',padding='same'))
classifier2.add(MaxPooling2D(pool_size=(2, 2)))

classifier2.add(MaxPooling2D(pool_size=(2, 2)))

classifier2.add(UpSampling2D(size=(2, 2)))
classifier2.add(Conv2DTranspose(64, (3, 3), strides=(1, 1), padding='same',activation='PReLU'))

# Add PReLU activation
        # classifier.add(PReLU())
        
# Add another Deconvolutional (Conv2DTranspose) layer to upsample the feature maps
classifier2.add(Conv2DTranspose(32, (3, 3), strides=(1, 1), padding='same',activation='PReLU'))

# Add PReLU activation
        # model.add(PReLU())

# Add another Deconvolutional (Conv2DTranspose) layer to upsample the feature maps
classifier2.add(Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='sigmoid'))
lr = 0.003
adam0 = Adam(learning_rate=lr)
classifier2.compile(optimizer=adam0, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
Evaluate1 = classifier2.fit(train_Image, epochs=10, validation_data=test_Image, batch_size=Batch_size1)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\dhruv\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\dhruv\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\dhruv\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\dhruv\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\dhruv\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D,UpSampling2D, Conv2DTranspose
from keras.optimizers import Adam

input_image_array_2 = input_image_array.reshape(1, rows, cols, channels)
print(input_image_array_2)
def build_classifier(input_shape):
    def bm():
        classifier = Sequential() # initialize neural network
        # Convolutional layers
        classifier.add(Conv2D(3, kernel_size=(3,3),strides=(1, 1), input_shape=input_shape, activation='PReLU',padding='same'))
        #classifier.add(MaxPooling2D(pool_size=(2, 2)))
        
        classifier.add(Conv2D(35, kernel_size=(3,3),strides=(1, 1), activation='PReLU',padding='same'))
        #classifier.add(MaxPooling2D(pool_size=(2, 2)))
        
        classifier.add(Conv2D(35, kernel_size=(3,3),strides=(1, 1), activation='PReLU',padding='same'))
        classifier.add(MaxPooling2D(pool_size=(2, 2)))

        classifier.add(MaxPooling2D(pool_size=(2, 2)))

        classifier.add(UpSampling2D(size=(2, 2)))
        classifier.add(Conv2DTranspose(64, (3, 3), strides=(1, 1), padding='same',activation='PReLU'))

# Add PReLU activation
        # classifier.add(PReLU())
        
# Add another Deconvolutional (Conv2DTranspose) layer to upsample the feature maps
        classifier.add(Conv2DTranspose(32, (3, 3), strides=(1, 1), padding='same',activation='PReLU'))

# Add PReLU activation
        # model.add(PReLU())

# Add another Deconvolutional (Conv2DTranspose) layer to upsample the feature maps
        classifier.add(Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='sigmoid'))
        
        
        
        # # Flatten layer
        # classifier.add(Flatten())
        
        # Fully connected layers
        #classifier.add(Dense(units=80, activation='relu'))
        #classifier.add(Dense(units=448, activation='relu'))
    
    
        # Compile the model
        lr = 0.003
        adam0 = Adam(learning_rate=lr)
        classifier.compile(optimizer=adam0, loss='categorical_crossentropy', metrics=['accuracy'])
        

# Predict the output
        output_image =classifier.predict(input_image_array_2)
        print(output_image.shape[3])
# Convert output to image
        output_image = array_to_img(output_image[0])

# Save the output image
        output_image.save("converted.jpg")
        return classifier
    return bm

# Usage example



In [ ]:
input_shape = (rows, cols, channels)
classifier = build_classifier(input_shape)()
print("a",classifier)

In [ ]:
print(classifier.summary())